In [1]:
import re
import numpy as np
from copy import copy
from operator import itemgetter

DATA

In [2]:
test_mapper = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

In [3]:
f = open("inputs/aoc5.txt").read()

# PART 1

In [4]:
def parse_seeds(f):
    l = f.splitlines()
    seeds = l[0].split(":")[1].strip().split(" ")
    seeds = [int(s) for s in seeds]
    return seeds
    

In [5]:
def parse_data(f):
    
    l = f.splitlines()

    headers = [i for i in l if "map" in i]

    header_start_indices = []
    for h_i, h in enumerate(headers):
        for i,v in enumerate(l):
            if h==v:
                header_start_indices.append(i+1)
    
    header_end_indices = copy(header_start_indices)
    header_end_indices.pop(0)
    header_end_indices.append(len(l))
    data = {}
    for i, h in enumerate(headers):
        s=header_start_indices[i]
        e=header_end_indices[i]-2 if i<(len(headers)-1) else header_end_indices[i]
        subset = l[s:e]
        # split
        subset = [i.split(" ") for i in subset]
        # to int
        subset = [[int(j) for j in i] for i in subset]
        data[h] = sorted(subset, key=itemgetter(1))
        
    return data

In [6]:
def source_to_destination_mapper(mapper_data, source_value):
    
    # load & parse
    for i,rng in enumerate(mapper_data):
        if (source_value>=rng[1]) and (source_value<rng[1]+rng[2]):
            return source_value - (rng[1]-rng[0])
        else:
            continue
    return source_value    

In [7]:
def total_mapper(all_mapper_data, source_value_seed):
    
    source_value = copy(source_value_seed)
    for k, mapper_data in all_mapper_data.items():
        new_source_value = source_to_destination_mapper(mapper_data, source_value)
        msg = f"{k} {source_value} -> {new_source_value}"

        source_value = new_source_value
    return source_value

## TEST

In [8]:
test_seeds = parse_seeds(test_mapper)
test_data = parse_data(test_mapper)

In [9]:
min_location = 1e20

for seed in test_seeds:

    location_value = total_mapper(test_data, seed)
    if location_value<min_location:
        min_location = location_value
print(min_location)  

35


## REAL DATA

In [10]:
seeds = parse_seeds(f)
data = parse_data(f)

In [11]:
min_location = 1e20

for seed in seeds:

    location_value = total_mapper(data, seed)
    if location_value<min_location:
        min_location = location_value
print(min_location)  

318728750


# Part 2

In [13]:
def range_intersection(range1, range2):
    start = max(range1[0], range2[0])
    end = min(range1[1], range2[1])
    
    # Check if there is a valid intersection
    if start <= end:
        return (start, end)
    else:
        return None

In [14]:
def subtract_ranges(list1, list2):
    result = []
    
    for start1, end1 in list1:
        current = start1
        for start2, end2 in list2:
            if current < start2:
                result.append((current, min(end1, start2 - 1)))
            current = max(current, end2 + 1)
        
        if current <= end1:
            result.append((current, end1))

    # Filter out empty ranges
    result = [(start, end) for start, end in result if start <= end]
    
    return result

# Example usage:
list1 = [(4, 25)]
list2 = [(3, 12), (20, 22)]
result = subtract_ranges(list2, list1)
print(result)


[(3, 3)]


In [15]:
i = (12,80)
s = (54,97)

In [16]:
range_intersection(i,s)

(54, 80)

In [17]:
def source_to_destination_mapper_ranges(mapper_data_lst, input_ranges: tuple):
    
    non_mapped = []
    intersection_ranges = []
    dest_ranges = []
        
    list_of_source_ranges = []
    # generate the destination values for overlapping ranges
    for input_range in input_ranges:
        for i,source_data in enumerate(mapper_data_lst):

                # make a source range
                interval_length = source_data[2]
                source_range = (source_data[1], source_data[1]+source_data[2]-1)        
                list_of_source_ranges.append(source_range)
                delta = source_data[1]-source_data[0]

                # check what has intersection / difference
                mapped_range = range_intersection(input_range, source_range)

                if mapped_range is not None:
                    intersection_ranges.append(mapped_range)
                    dest_ranges.append((mapped_range[0]-delta,mapped_range[1]-delta))
        #print("intersection:", intersection_ranges)
        #print("Destination value of intersection:", dest_ranges)
        # numbers in input_Range but not in list of source_ranges, get mapped to same value

        non_mapped_data = subtract_ranges([input_range],list_of_source_ranges)
        #print("non-mapped:", non_mapped_data)
        dest_ranges.extend(non_mapped_data)
        
    return dest_ranges

In [38]:
def map_all(mapper_data_dict, input_ranges):
    
    minimum = 1e20
    
    dest_ranges = input_ranges
    for k, mapper_data_lst in mapper_data_dict.items(): 
        print(k)
        print("source ranges: ", dest_ranges)
        dest_ranges = source_to_destination_mapper_ranges(mapper_data_lst, dest_ranges)
        print("dest ranges", dest_ranges)
        print()
            
    for dest_range in dest_ranges:
        if min(dest_range)<minimum:
            minimum = min(dest_range)
    return minimum
            

## Test

In [43]:
test_seed_ranges = [(55,55+13-1), (79,79+14-1)]
test_data = parse_data(test_mapper)

In [44]:
test_seed_ranges

[(55, 67), (79, 92)]

In [45]:
test_data

{'seed-to-soil map:': [[52, 50, 48], [50, 98, 2]],
 'soil-to-fertilizer map:': [[39, 0, 15], [0, 15, 37], [37, 52, 2]],
 'fertilizer-to-water map:': [[42, 0, 7],
  [57, 7, 4],
  [0, 11, 42],
  [49, 53, 8]],
 'water-to-light map:': [[88, 18, 7], [18, 25, 70]],
 'light-to-temperature map:': [[81, 45, 19], [68, 64, 13], [45, 77, 23]],
 'temperature-to-humidity map:': [[1, 0, 69], [0, 69, 1]],
 'humidity-to-location map:': [[60, 56, 37], [56, 93, 4]]}

In [46]:
map_all(test_data, test_seed_ranges)

seed-to-soil map:
source ranges:  [(55, 67), (79, 92)]
dest ranges [(57, 69), (81, 94)]

soil-to-fertilizer map:
source ranges:  [(57, 69), (81, 94)]
dest ranges [(57, 69), (81, 94)]

fertilizer-to-water map:
source ranges:  [(57, 69), (81, 94)]
dest ranges [(53, 56), (61, 69), (81, 94)]

water-to-light map:
source ranges:  [(53, 56), (61, 69), (81, 94)]
dest ranges [(46, 49), (54, 62), (74, 87)]

light-to-temperature map:
source ranges:  [(46, 49), (54, 62), (74, 87)]
dest ranges [(82, 85), (90, 98), (78, 80), (45, 55)]

temperature-to-humidity map:
source ranges:  [(82, 85), (90, 98), (78, 80), (45, 55)]
dest ranges [(82, 85), (90, 98), (78, 80), (46, 56)]

humidity-to-location map:
source ranges:  [(82, 85), (90, 98), (78, 80), (46, 56)]
dest ranges [(86, 89), (94, 96), (56, 59), (97, 98), (82, 84), (60, 60), (46, 55)]



46

In [47]:
def parse_seeds2(f):
    
    l = f.splitlines()
    seeds = l[0].split(":")[1].strip().split(" ")
    seeds = [int(s) for s in seeds]
    
    new_seeds = []
    for idx in np.arange(0,len(seeds),2):
        seed_start_nr = seeds[idx]
        new_seeds.append((seed_start_nr,seed_start_nr+seeds[idx+1]-1))
    
    return sorted(new_seeds, key=itemgetter(0))  
        

In [48]:
seeds2 = parse_seeds2(f)

In [49]:
# lowest and highest number of seed numbers to consider
seeds2

[(202517468, 334158438),
 (460203450, 544834348),
 (856311572, 1399051680),
 (1435322022, 1535691088),
 (1553776977, 1795605556),
 (1809826083, 1962970235),
 (2019100043, 2172806598),
 (2494032210, 2729189393),
 (2797169753, 2974686908),
 (3766866638, 3881127744)]

In [50]:
map_all(data, seeds2)

seed-to-soil map:
source ranges:  [(202517468, 334158438), (460203450, 544834348), (856311572, 1399051680), (1435322022, 1535691088), (1553776977, 1795605556), (1809826083, 1962970235), (2019100043, 2172806598), (2494032210, 2729189393), (2797169753, 2974686908), (3766866638, 3881127744)]
dest ranges [(246837326, 378478296), (1815925015, 1900555913), (1632469937, 1787834567), (2044594803, 2059837852), (1233103806, 1265974897), (943241642, 1059486393), (1059486394, 1086566451), (2000660015, 2025282600), (476432861, 609280126), (1173097443, 1211564116), (672016787, 772385853), (790471742, 840113386), (0, 460282), (1553049016, 1585337375), (1357621124, 1393363308), (460283, 124156389), (138376916, 169037771), (609280127, 612853359), (2025282601, 2044594802), (1287564557, 1357621123), (1086566452, 1116107746), (1172237554, 1173097442), (612853360, 657286134), (2064392707, 2172806598), (2796989323, 2810085326), (3398847262, 3540020131), (3381542286, 3398847261), (2343571960, 2407155293), (2

37384986